# <span style="color:DarkRed"> Neural Network by scratch </span>

## Info

Κ : targets/categories/class  (πλήθος κατηγοριών)

N : all training data   (πλήθος δεδομένων εισόδου)

M : hidden units        (ένα κρυφό επίπεδο με  hidden_layer_size = Μ)

$W^{(1)} $πίνακας βαρών με μέγεθος: $Μ x (D + 1)$

$W^{(2)} $πίνακας βαρών με μέγεθος: $K x M + 1$

***

![title](Image/myMLP.png)

Image From:https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6 (for 2 layers MLP)

Mapping with my MLP:

- σ = h :Activation Function

- Loss = Cost Function 

***

LaTex for maths in notebook
https://www.math.ubc.ca/~pwalls/math-python/jupyter/latex/

***

## Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
#need for cifar dataset
import pickle

***

## Activations Functions 

### {non linear function}

Activation function for hidden layer:

 $$h(.) = **Activation Function**$$

(1)softplus function:$$h(α) = log(1 + e^α)$$ 

(2)$$ h(α) = \frac{e^α− e^{−α}}{e^α + e^{−α}} = \frac {\frac{e^α− e^{−α}}{2}}{\frac{e^α + e^{−α}}{2}}= \frac{sin(a)}{cos(a)}= tanh(a) $$

(3)$$ h(α) = cos(α)(=\frac{e^α + e^{−α}}{2})$$

In [3]:
def activationLog(a):
    h = np.log( 1 + np.exp(a))
    return h

In [4]:
def activationDiv(a):
    #h =  (np.exp(a) - np.exp(-a)) / (np.exp (a) + np.exp(-a)) //Not safe-overflow
    #TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
    h = np.tanh(a)
    return h

In [5]:
def activationCos(a):
    h =  np.cos(a)
    return h

In [6]:
def activation(activID,a):
    """
    Execute the activation on input data
    :param activID: The id of the activation we choose
    :param a: Out input that will pass through the activation

    :return: The result of the activation
    """

    if activID=='1':
        h = activationLog(a)
    elif activID=='2':
        h = activationDiv(a)
    elif activID=='3':
        h = activationCos(a)
    else : print("you don't give number 1,2 or 3")

    return h

Activation Function for output layer:

$$softmax(a_i) = \frac{ e^{a_i} }{ \sum_{i=1}^{K} e^{ a_i } }$$

In [7]:
#use by default ax=1, when the array is 2D
#use ax=0 when the array is 1D
def activationSoftmax( z, ax=1):
    m = np.max( z, axis=ax, keepdims=True )#max per row
    p = np.exp( z - m )
    return ( p / np.sum(p,axis=ax,keepdims=True) )

***

# Model Structure

## Layer

### {linear function}

Dense Layer

$$f(x ; w)= W^{T} X$$

Hidden layer:
{activationFunct: no linear function} with {f : linear function} => {z : no linear function}

$$ z = h(f) = h ( W^{T} X) $$

***

Output layer: use activation function **softmax function** (input is the output from hidden layer)

$$softmax(a_i) = \frac{ e^{a_i} }{ \sum_{i=1}^{K} e^{ a_i } }$$

$$=> y_{nk} = \frac{ e^{(w_{k}^{(2)})^{T} z_{n}} }{ \sum_{j=1}^{K} e^{(w_{j}^{(2)})^{T} z_{n} } }$$

In [8]:
def feedForward(W1, W2, X, activID, lamda, t):

    # Hidden layer
    f1 = X.dot(W1.T)
    Z = activation(activID, f1)

    # add bias #Append a column with aces at the start
    ones = np.ones((Z.shape[0], 1))
    Z = np.append(ones, Z, axis=1)

    # Output layer
    f2 = Z.dot(W2.T)
    max_error = np.max(f2, axis=1)

    Y = activationSoftmax(f2)


    # cost function #mathematical analysis below
    Cost = np.sum(t * f2) - np.sum(max_error) - \
           np.sum(np.log(np.sum(np.exp(f2 - np.array([max_error, ] * f2.shape[1]).T), 1))) - \
           ((1 / 2) * lamda) * (np.sum(np.square(W1)) + np.sum(np.square(W2)))


    return Z, Y, Cost

(logLikelihood plus reguralization term)

we want to maximize for the problem of classifying N number of data in K categories/classes is:

$$
E(W) = \sum_{n=1}^N \sum_{k=1}^K t_{nk} \log y_{nk}   -  \frac{\lambda}{2} \sum_{k=1}^K ||\mathbf{w_k}||^2 
$$

$$
=> E(W) = \sum_{n=1}^N \sum_{k=1}^K t_{nk} \log (\frac{e^{\mathbf{w}_k^T \mathbf{z}_n}}{\sum_{j=1}^K e^{\mathbf{w}_j^T \mathbf{z}_n}})   -  \frac{\lambda}{2} \sum_{k=1}^K ||\mathbf{w_k}||^2
$$

$$
=>E(W) = \sum_{n=1}^N \left[  \sum_{k=1}^K t_{nk} \log (e^{\mathbf{w}_k^T \mathbf{z}_n})  - \log \left( \sum_{j=1}^K e^{\mathbf{w}_j^T \mathbf{z}_n} \right) \right]   -  \frac{\lambda}{2} \sum_{k=1}^K ||\mathbf{w}_k||^2
$$

$$
=> E(W) = \sum_{n=1}^N \left[ \left( \sum_{k=1}^K t_{nk} \mathbf{w}_k^T \mathbf{z}_n \right) - \log \left( \sum_{j=1}^K e^{\mathbf{w}_j^T \mathbf{z}_n} \right) \right]   -  \frac{\lambda}{2} \sum_{k=1}^K ||\mathbf{w}_k||^2, 
$$

where $y_{nk}$ is the softmax function defined as:

$$y_{nk} = \frac{e^{\mathbf{w}_k^T \mathbf{z}_n}}{\sum_{j=1}^K e^{\mathbf{w}_j^T \mathbf{z}_n}}$$
$W$ is a $K \times (D+1)$ matrix where each line represents the vector $\mathbf{w}_k$.


The cost function can be simplified in the following form:



$$
E(W) = \sum_{n=1}^N \left[ \left( \sum_{k=1}^K t_{nk} \mathbf{w}_k^T \mathbf{z}_n \right) - \log \left( \sum_{j=1}^K e^{\mathbf{w}_j^T \mathbf{z}_n} \right) \right]   -  \frac{\lambda}{2} \sum_{k=1}^K ||\mathbf{w}_k||^2, 
$$



In the above formula we have used the fact that $\sum_{k=1}^K t_{nk} = 1$. 

The partial derrivatives of this function are given by the following $K \times (D+1)$ matrix:

$$
(T - Y)^Τ Z - \lambda W,
$$

**!όλοι αυτή η συνάρτηση ΕΙΝΑΙ Η ΜΕΡΙΚΗ ΠΑΡΑΓΩΓΟΣ της συναρτησεις κόστους!**

where:

$T$ is an $N \times K$ matrix with the truth values of the training data, such that $[T]_{nk} = t_{nk}$, 

$Y$ is the corresponding $N \times K$ matrix that holds the softmax probabilities such that $[Y]_{nk} = y_{nk}$, 

$Z$ is the $N \times (D + 1)$ matrix of the input data

we use the logsumexp trick, where m is the maximum element
\begin{align} 
\log \sum_{i=1}^{n} e^{\mathbf{w}_j^T \mathbf{z}_n} &= \log \Bigr( \sum_{i=1}^{n} e^{\mathbf{w}_j^T \mathbf{z}_n +m -m}\Bigl) \\ 
&= \log \Bigr( \sum_{i=1}^{n} e^m e^{\mathbf{w}_j^T \mathbf{z}_n-m} ) \Bigl) \\ 
&= \log \Bigr( e^m \sum_{i=1}^{n} e^{\mathbf{w}_j^T \mathbf{z}_n-m} ) \Bigl) \\ 
&= \log \ e^m + \log \Bigr( \sum_{i=1}^{n} e^{\mathbf{w}_j^T \mathbf{z}_n-m} ) \Bigl) \\ 
&= m + \log \Bigr( \sum_{i=1} e^{\mathbf{w}_j^T \mathbf{z}_n-m}  \Bigl) 
\end{align}

***

## BACKPROGATION(Stochastic gradient ascent)

### Update weights

In [9]:
def updateWeights(W, lr, grad):
    
    W = W + lr * grad
    
    return W

### Partial Derivatives (Μερικοί Παράγωγοι)

**Weights layer 2 (output)**

Partial Derivative for Cost Function (ΜΕΡΙΚΗ ΠΑΡΑΓΩΓΟΣ της συναρτησεις κόστους)

EN: Some derivatives for the parameters W (2) have a form similar to that of simple linear logistic regression of many categories and are given by the relation:

GR: Οι μερικές παράγωγοι για τις παραμέτρους W (2) έχουν όμοια μορφή με αυτή της απλής γραμμικής λογιστικής παλινδρόμησης πολλών κατηγοριών και δίνονται από την σχέση:

$$
\frac{\partial Cost}{\partial{W_2}} =(T - Y)^Τ Z - \lambda W_2,
$$

In [10]:
def gradientW2(W2 , Y, Z , T ,lamda):
    # calculate gradient
    gradientW2 = (T - Y).T.dot(Z) - lamda * W2
    return gradientW2

**Weights layer 1 (hidden)**

Partial Derivative for Activation functions:

(1)$$h(α) = log(1 + e^α)$$

$$\frac{\partial h(α)}{\partial{a}} = \frac{\partial(log(1 + e^α))}{\partial{α}}$$
Apply chain rule 

(rememder $(log(x))'= \frac {1}{1 + x}$):
$$ = \frac {1}{1 + e^a} (1 + e^a)'$$

$$= \frac {1}{1 + e^a} e^a$$

$$= \frac {e^a}{1 + e^a} $$

$$= \frac {e^a}{e^a(\frac {1}{e^a} + 1)} $$

$$= \frac {1}{\frac {1}{e^a} + 1} $$
$$= \frac {1}{e^{-a} + 1} $$

In [11]:
def derivativeLog(a):
    d = 1 / ( np.exp(-a) + 1 )
    return d

(2)$$ h(α) = \frac{e^α− e^{−α}}{e^α + e^{−α}} $$

$$\frac{\partial h(α)}{\partial{α}} = (\frac{e^α− e^{−α}}{e^α + e^{−α}})'$$
$$= \frac {(e^α − e^{−α})'(e^α + e^{−α}) - (e^α − e^{−α})(e^α + e^{−α})'}{(e^α + e^{−α})^2}$$
$$= \frac {(e^α + e^{−α})(e^α + e^{−α}) - (e^α − e^{−α})(e^α - e^{−α})}{(e^α + e^{−α})^2}$$
$$= \frac {(e^α + e^{−α})^2 - (e^α − e^{−α})^2}{(e^α + e^{−α})^2}$$
$$= 1-\frac {(e^α − e^{−α})^2}{(e^α + e^{−α})^2}$$
$$= 1-(\frac {e^α − e^{−α}}{e^α + e^{−α}})^2$$
$$= 1- tanh(a)^2$$

In [12]:
def derivativeDiv(a):
    #d = 4 / ((np.exp(a) + np.exp(-a))^2)//Not safe-overflow
    d = 1 - np.power(np.tanh(a), 2)
    return d

(3)$$ h(α) = cos(α)$$

$$(cos(α))' = - sin(a)$$

In [13]:
def derivativeCos(a):
    d = - np.sin(a)
    return d

In [14]:
def derivate_activation(activID, a):
    """
    Execute the activation on input data
    inputs :
    :param activID: The id of the activation we choose
    :param a: Out input that will pass through the deravate activation

    output:
    :return: The result of the derivate of activation
    """

    if activID == '1':
        dh = derivativeLog(a)
    elif activID == '2':
        dh = derivativeDiv(a)
    elif activID == '3':
        dh = derivativeCos(a)
    else:
        print("you don't give number 1,2 or 3")

    return dh

Reminder: 
<table style="width:50%">
  <tr>
    <th>//Hidden Layer</th>
    <th> //Output Layer</th>
    
  </tr>
  <tr>
    <td>$f1=W1^{T}X$</td>
    <td>$f2=W2^{T}Z$</td>
    
  </tr>
  <tr>
    <td>$Z = h(f1)$</td>
    <td>$Y = softmax(f2)$</td>
    
  </tr>
</table>

$\frac{\partial Cost}{\partial{Z}} = \frac{\partial Cost}{\partial{Y}} * \frac{\partial Y}{\partial{f2}} * \frac{\partial f2}{\partial{Z}}$ (chain rule)&nbsp;&nbsp;&nbsp;  $[\frac{\partial f2}{\partial{Z}}= W2]$

$\frac{\partial z}{\partial{f1}} = eg.-sin(f1)$&nbsp;(2)&nbsp;//derevate of activation

$\frac{\partial f1}{\partial{W1}} = X $&nbsp;(3)&nbsp;//derevate of linear

=> $ \frac{\partial Cost}{\partial{W1}} = \frac{\partial Cost}{\partial{Z}} * \frac{\partial z}{\partial{f1}} * \frac{\partial f1}{\partial{W1}}$&nbsp;(4) &nbsp;(chain rule)


So, we have

$\frac{\partial Cost}{\partial{Z}} = (T-Y) * W2 $ (1)

(1)*(2) $\frac{\partial Cost}{\partial{Z}} * \frac{\partial z}{\partial{f1}} = ((T-Y) * W2)* (eg.-sin (f1))$&nbsp;(3)

(1)*(2)*(3) $\frac{\partial Cost}{\partial{Z}} * \frac{\partial z}{\partial{f1}} * \frac{\partial f1}{\partial{W1}} = ((T-Y)* W2)^{T}* (eg.-sin (f1))* X - λW1$ &nbsp;(4)

In [15]:
def gradientW1(W1, W2, X, activID, lamda, T, Y):
    # Remove the bias
    w2_noBias = np.copy(W2[:, 1:])

    f1 = X.dot(W1.T)
    dz = derivate_activation(activID, f1)

    dc = (T - Y).dot(w2_noBias)

    gradiendW1 = (dc * dz).T.dot(X) - lamda * W1

    return gradiendW1

***

### Gradient check

EN: The function `gradCheck` compare the exact gradients with 
numerical estimates obtained by finite differences

GR: Η συνάρτηση `gradCheck` που συγκρίνει τις αναλυτικές παραγώγους με αριθμητικές διαφορές

(μέθοδος για να σιγουρευτούμε ότι οι παραγωγει που υπολογίσαμε είναι σωστές)

$$\frac{\partial E}{\partial{w_ij}}= \frac{E(w_{ij}+ε)-E(w_{ij}-ε)}{2ε}$$

$E= Cost$ 

and 

$ε=epsilon= 10^{-6}=1e-6$

In [16]:
def gradCheck(W1, W2, X, t, lamda, activID):
    """
    input:
    :param W1: Initial weights of Hidden Layer
    :param W2: Initial weights of Output Layer
    :param X: Our data
    :param t: The corresponding labels
    :param lamda:(regularization term)
    :param activID: The id of the activation we choose
    """

    epsilon = 1e-6

    onesX = np.ones((X.shape[0], 1))
    X = np.append(onesX, X, axis=1)

    _list = np.random.randint(X.shape[0], size=5)
    x_sample = np.array(X[_list, :])
    t_sample = np.array(t[_list, :])

    # Feed Forward
    Z, Y, Cost = feedForward(W1, W2, x_sample, activID, lamda, t_sample)

    # Backprogation # Stocastic gradient
    gradW2 = gradientW2(W2, Y, Z, t_sample, lamda)
    gradW1 = gradientW1(W1, W2, x_sample, activID, lamda, t_sample, Y)

    numericalGrad = np.zeros(gradW1.shape)
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad
    for k in range(numericalGrad.shape[0]):
        for d in range(numericalGrad.shape[1]):
            # add epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] += epsilon
            _, _, e_plus = feedForward(w_tmp, W2, x_sample, activID, lamda, t_sample)  # Feed Forward

            # subtract epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] -= epsilon
            _, _, e_minus = feedForward(w_tmp, W2, x_sample, activID, lamda, t_sample)  # Feed Forward

            # approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad[k, d] = (e_plus - e_minus) / (2 * epsilon)

    # Absolute norm
    print("The difference estimate for gradient of w1 is : ", np.max(np.abs(gradW1 - numericalGrad)))

    numericalGrad = np.zeros(gradW2.shape)
    # Compute all numerical gradient estimates and store them in
    # the matrix numericalGrad
    for k in range(numericalGrad.shape[0]):
        for d in range(numericalGrad.shape[1]):
            # add epsilon to the w[k,d]
            w_tmp = np.copy(W2)
            w_tmp[k, d] += epsilon
            _, _, e_plus = feedForward(W1, w_tmp, x_sample, activID, lamda, t_sample)  # Feed Forward

            # subtract epsilon to the w[k,d]
            w_tmp = np.copy(W2)
            w_tmp[k, d] -= epsilon
            _, _, e_minus = feedForward(W1, w_tmp, x_sample, activID, lamda, t_sample)  # Feed Forward

            # approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad[k, d] = (e_plus - e_minus) / (2 * epsilon)
    # Absolute norm
    print("The difference estimate for gradient of w2 is : ", np.max(np.abs(gradW2 - numericalGrad)))
    
    

**Μore detail**

During gradient ascent/descent we compute the gradients $\frac{\partial E}{\partial w}$, where $w$ denotes the parameters of the model.

In order to make sure that these gradients are correct we will compare the exact gradients(that we have coded) with numerical estimates obtained by finite differences, you can use your code for computing $E$ to verify the code for computing $\frac{\partial E}{\partial w}$.
    Let's look back at the definition of a derivative (or gradient):
    
$$ \frac{\partial E}{\partial w} = \lim_{\varepsilon \to 0} \frac{E(w + \varepsilon) - E(w - \varepsilon)}{2 \varepsilon} \tag{1}$$  

We know the following: 
- $\frac{\partial E}{\partial w}$ is what you want to make sure you're computing correctly. ,
- You can compute $E(w + \varepsilon)$ and $E(w - \varepsilon)$ (in the case that $w$ is a real number), since you're confident your implementation for $E$ is correct.

Let's use equation (1) and a small value ( around $10^-4$ or $10^-6$, much smaller values could lead to numerical issues )for $\varepsilon$ to make sure that your code for computing  $\frac{\partial E}{\partial w}$ is correct!

![title](Image/grad.png)

***

# Train Model

In [17]:
def train(W1, W2, x_train, y_train, lr, epochs, batch_size, lamda, activID):
    """
    Training process

    inputs :
    :param w1: Hidden Layer weights
    :param w2: Output Layer weights

    :param x_train: Table for training data N x D . N: number of data, D: dimension
    :param y_train: Table for training labels N x d

    :param lr: learning rate

    :param epochs: number of epochs
    :param  batch_size: batch size

    :param lamda: value of lamda (regularization term)

    :param activID: The id of the activation we choose

    output :
    :return: Trained weight of the network to use them for predictions
    """

    # add bias
    # add column of ones to the dataset for multiply in linear function: 1* w0
    # f(x)= 1*w0 + x*w1...
    onesArr = np.ones((x_train.shape[0], 1))
    X_train = np.append(onesArr, x_train, axis=1)

    # decay learning rate
    # towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
    lr = lr / epochs

    for i in range(epochs):

        print("Epoch:" + str(i + 1) + '/' + str(epochs))

        # zip the feature and the labels, so shuffle with same way
        connectArray = list(zip(X_train, y_train))

        # shuffle: the same examples avoid to be in the same batches
        np.random.shuffle(connectArray)

        X_train, y_train = zip(*connectArray)
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        # Train of each batch
        for j in range(0, X_train.shape[0], batch_size):
            batch_x = X_train[j: j + batch_size, :]  # is X
            batch_y = y_train[j: j + batch_size, :]  # is T

            # Neural Network:

            # Feed Forward
            Z, Y, Cost = feedForward(W1, W2, batch_x, activID, lamda, batch_y)

            # Backprogation # Stocastic gradient
            W2_grad = gradientW2(W2, Y, Z, batch_y, lamda)
            W1_grad = gradientW1(W1, W2, batch_x, activID, lamda, batch_y, Y)

            W1 = updateWeights(W1, lr, W1_grad)
            W2 = updateWeights(W2, lr, W2_grad)

    return W1, W2


***

# Load Dataset

Here is a python3 routine which will open such a file and return a dictionary of cifar dataset:
    https://www.cs.toronto.edu/~kriz/cifar.html

In [18]:
import pickle
def unpickle(file):
    """
    Here is a python3 routine which will open such a file and return a dictionary:
    https://www.cs.toronto.edu/~kriz/cifar.html
    
    input:
    :param file: Path of dataset
    output:
    :return: Dictionary to dataset
    """
    with open(file, 'rb') as fo:
        dictionary = pickle.load(fo, encoding='bytes')
    return dictionary

### One hot vector

Categorical element:

For each element in dataset belong to only one target k.

So we have a binary vector of length K.

$t_{nk} \in \{ 0,1 \}, \sum_{k=1}^{K} t_{nk} = 1$

***

In [19]:
def loadDataset(datasetID):
    """
    inputs :
    :param datasetID: Id of the dataset thato user want to load
                        1 -> mnist or 2 -> cifar
    """
    x_train = []
    x_test = []
    y_train = []
    y_test = []

    if datasetID == '1':
        # load the train files
        df = None
        y_train = []
        for i in range(10):

            tmp = pd.read_csv('Data/mnist/train%d.txt' % i, header=None, sep=" ")

            # one hot vector for labels (y_train)
            hot_vector = [1 if j == i else 0 for j in range(0, 10)]
            for j in range(tmp.shape[0]):
                y_train.append(hot_vector)

            # concatenate dataframes by rows
            # concat all train txt from mnist
            if i == 0:
                df = tmp
            else:
                df = pd.concat([df, tmp])

        x_train = df.as_matrix()
        y_train = np.array(y_train)

        # load test files
        df = None
        y_test = []
        for i in range(10):

            tmp = pd.read_csv('Data/mnist/test%d.txt' % i, header=None, sep=" ")

            # one hot vector for labels (y_test)
            hot_vector = [1 if j == i else 0 for j in range(0, 10)]
            for j in range(tmp.shape[0]):
                y_test.append(hot_vector)

            # concatenate dataframes by rows
            # concat all test txt from mnist
            if i == 0:
                df = tmp
            else:
                df = pd.concat([df, tmp])

        x_test = df.as_matrix()
        y_test = np.array(y_test)

        # tranform the data of image to scale[0,1]
        x_train = x_train.astype(float) / 255
        x_test = x_test.astype(float) / 255

        return x_train, x_test, y_train, y_test

    elif datasetID == '2':
        # load the train files
        for i in range(5):
            path = 'Data/cifar-10-batches-py/data_batch_{0:d}'.format(i+1)
            batch_i = unpickle(path)

            for j in range(len(batch_i['data'.encode('ascii', 'ignore')])):
                x_train.append(batch_i['data'.encode('ascii', 'ignore')][j])
                labels = batch_i['labels'.encode('ascii', 'ignore')][j]

                # one hot vector for labels (y_train)
                hot_vector = [1 if k == labels else 0 for k in range(0, 10)]
                y_train.append(hot_vector)

        # load the test file
        path = 'Data/cifar-10-batches-py/test_batch'
        test_batch = unpickle(path)

        for j in range(len(test_batch['data'.encode('ascii', 'ignore')])):
            x_test.append(test_batch['data'.encode('ascii', 'ignore')][j])
            labels = test_batch['labels'.encode('ascii', 'ignore')][j]

            # one hot vector for labels (y_test)
            hot_vector = [1 if k == labels else 0 for k in range(0, 10)]
            y_test.append(hot_vector)

    return np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)


***

# Run

## Initialize

In [20]:
datasetID = input("\nSelect a dataset id: \n"
                        "id : dataset \n"
                        "-------------\n"
                        "1  : MNIST \n"
                        "2  : CIFAR-10 \n"
                        "Type here: ")


Select a dataset id: 
id : dataset 
-------------
1  : MNIST 
2  : CIFAR-10 
Type here: 1


In [21]:
if datasetID == '2':
    x_train, x_test, y_train, y_test = loadDataset(datasetID)
else:
    if datasetID != '1':
        print("By default selected the MNIST dataset")
    x_train, x_test, y_train, y_test = loadDataset(datasetID)

C:\Users\Connalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Connalia\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


***

In [22]:
activID = input("\nSelect a activation function id for hidden layer: \n"
                        "id:  activation \n"
                        "----------------\n"
                        "1 : log(1+exp(a))\n"
                        "2 :(exp(a)-exp(-a))/((a)+exp(-a))\n"
                        "3 : cos(a) \n"
                        "Type here: ")


Select a activation function id for hidden layer: 
id:  activation 
----------------
1 : log(1+exp(a))
2 :(exp(a)-exp(-a))/((a)+exp(-a))
3 : cos(a) 
Type here: 3


In [23]:
# default activation id
if activID != '1' and activID != '2' and activID != '3':
    print("By default selected the activation function cos(a)")
    activID = '3'

***

In [24]:
batch_sizeID = input("\nSelect size of minibatches: \n"
                         "id:  batch size \n"
                         "----------------\n"
                         "1 : 100\n"
                         "2 : 200\n"
                         "Type here: ")


Select size of minibatches: 
id:  batch size 
----------------
1 : 100
2 : 200
Type here: 1


In [25]:
# default size of minibatches
if batch_sizeID == '2':
    batch_size = 200
else:
    if batch_sizeID != '1':
        print("By default selected batch size = 100")
    batch_size = 100

***

In [26]:
M = input("\nSelect number of neurons in the hidden layer: \n"
              "id:  neurons \n"
              "----------------\n"
              "1 : 100\n"
              "2 : 200\n"
              "3 : 300\n"
              "Type here: ")


Select number of neurons in the hidden layer: 
id:  neurons 
----------------
1 : 100
2 : 200
3 : 300
Type here: 1


In [27]:
# default number of neurons
if M == '2':
    M = 200
elif M == '3':
    M = 300
else:
    if M != '1':
        print("By default selected 100 neurons")
    M = 100

***

In [28]:
# regularization parameter in CostFunction
lamda = 0.1

# learning rate
lr = 0.01

# number of epochs
epochs = 5

***

Mnists consists of $28x28$ grayscale images, so the image have $784$ pixels.

In total there are 10 training files (train0.txt, train1.txt, ..., train9.txt) where each rows of train$k$.txt corresponds to an example that belongs to the category $k$.

* So we have 10 targets/categories in mnist: 0-9


Cifar consists of $32x32$ images, so the image have $1024$ pixels.
* So we have 10 targets/categories in cifar: 1.airplane,2.automobile,3.bird,4.cat,5.deer,6.dog,7.frog,8.horse,9.ship,10.truck


In [29]:
x_train.shape

(60000, 784)

In [30]:
# num of categories
K = 10

# D: num of input pixels : eg mnist 28*28 = 784
# N: num of training examples
N, D = x_train.shape


# initialize weights

# follow normal distribution
center = 0
s = 1 / np.sqrt(D + 1)
W1 = np.random.normal(center, s, (M, D + 1))

# random init
W2 = np.random.rand(K, M + 1)

***

## Gradient check

In [31]:
gradCheck(W1, W2, x_train, y_train, lamda, activID)

The difference estimate for gradient of w1 is :  5.969499261571087e-08
The difference estimate for gradient of w2 is :  4.08211193736463e-08


***

## Training

In [32]:
W1_train, W2_train = train(W1, W2, x_train, y_train, lr, epochs, batch_size, lamda, activID)

Epoch:1/5
Epoch:2/5
Epoch:3/5
Epoch:4/5
Epoch:5/5


## Predict

`y,_ = function()` means that the function return two parameters,but we want only `y` 

(here predict=y)

In [33]:
# Add the bias on test data
onesArrTest = np.ones((x_test.shape[0], 1))
x_test = np.append(onesArrTest, x_test, axis=1)

_, predict, Cost = feedForward(W1_train, W2_train, x_test, activID, lamda, y_test)

## Accuracy

In [34]:
predict = np.argmax(predict, 1)
real = np.argmax(y_test, 1)

accurancy =np.mean( predict == np.argmax(y_test,1) )

print("Accurancy: " +str(accurancy))

Accurancy: 0.9728
